In [1]:
!pip install tensorflow_addons

In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, TimeDistributed, Activation, Bidirectional, Lambda, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

In [3]:
# Download and unzip dataset
import gdown

url = 'https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu'
output = 'all_wav.zip'
gdown.download(url, output, quiet=False)
!unzip -q 'all_wav.zip' -d '/content/all_wav'

url = 'https://drive.google.com/uc?id=1vqvn0F0YYhEFbzLgP9wJ36vyInUnO5b5'
output = 'dataset.csv'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu
From (redirected): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu&confirm=t&uuid=4ccab921-477b-46ce-9bfb-c0a20a520ac7
To: /content/all_wav.zip
  1%|          | 29.4M/2.48G [00:00<00:31, 77.6MB/s]

KeyboardInterrupt: 

In [3]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')
df.head()

,wav_filename,wav_filesize,transcript,confidence_level
0,./all_wav/Tehran_SayeRoshan0_101.wav,83044,اتفاقاتی که ندیده بودم,0.927557
1,./all_wav/Tehran_SayeRoshan0_105.wav,54468,مسجد,0.927557
2,./all_wav/Tehran_SayeRoshan0_107.wav,136036,جمع شدن مسلمین برای نمازهای جماعت,0.864152
3,./all_wav/Tehran_SayeRoshan0_108.wav,106788,همیشه برای محمدرضا پهلوی,0.927557
4,./all_wav/Tehran_SayeRoshan0_109.wav,170020,چه زمانی در کسوت شاه ایران نوکری اجانب را می‌کرد,0.854824


In [4]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')

# Update paths for WAV files
df['wav_filename'] = df['wav_filename'].apply(lambda x: x.replace('./all_wav/', '/content/all_wav/all_wav/'))

# Filter dataset to include only existing WAV files
df = df[df['wav_filename'].apply(os.path.isfile)]

# Character map for Persian characters
char_map_str = """
' 0
<SPACE> 1
ا 2
ب 3
پ 4
ت 5
ث 6
ج 7
چ 8
ح 9
خ 10
د 11
ذ 12
ر 13
ز 14
ژ 15
س 16
ش 17
ص 18
ض 19
ط 20
ظ 21
ع 22
غ 23
ف 24
ق 25
ک 26
گ 27
ل 28
م 29
ن 30
و 31
ه 32
ی 33
، 34
؟ 35
"""
char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)] = ch
index_map[1] = ' '

# Ensure space character is in char_map
char_map[' '] = char_map['<SPACE>']

# Audio processing functions
def load_audio(file_path, sr=16000):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

def extract_features(audio, n_mfcc=20, sr=16000):  # Reduce the number of MFCC features
    mfcc_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    delta_mfcc = librosa.feature.delta(mfcc_features)
    combined = np.vstack((mfcc_features, delta_mfcc)).T  # Use MFCC and delta only
    return combined

def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    augmented_audio = augmented_audio.astype(type(audio[0]))
    return augmented_audio

# Prepare the dataset
X = []
y = []
input_lengths = []
label_lengths = []

for index, row in df.iterrows():
    audio_path = row['wav_filename']
    audio = load_audio(audio_path)
    audio = add_noise(audio)  # Add noise
    features = extract_features(audio)
    X.append(features)
    input_lengths.append(features.shape[0])
    label = [char_map.get(c, char_map[' ']) for c in row['transcript']]
    y.append(label)
    label_lengths.append(len(label))

if len(X) == 0 or len(y) == 0:
    raise ValueError("No valid audio files were found. Please check the dataset and the paths.")

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding='post', value=-1)

# Define the model with reduced complexity
input_data = Input(name='the_input', shape=(None, 40))  # Updated input shape due to reduced features
masking_layer = Masking(mask_value=0.0)(input_data)
bilstm_layer_1 = Bidirectional(LSTM(128, return_sequences=True))(masking_layer)
batch_norm_1 = BatchNormalization()(bilstm_layer_1)
dropout_1 = Dropout(0.3)(batch_norm_1)
bilstm_layer_2 = Bidirectional(LSTM(128, return_sequences=True))(dropout_1)
batch_norm_2 = BatchNormalization()(bilstm_layer_2)
dropout_2 = Dropout(0.3)(batch_norm_2)
time_dense = TimeDistributed(Dense(len(char_map) + 1))(dropout_2)
y_pred = Activation('softmax', name='activation')(time_dense)

# Define the CTC loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Compile the model with CTC loss
labels = Input(name='the_labels', shape=[None], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'ctc': lambda y_true, y_pred: y_pred})

# Split the data into training and validation sets
X_train, X_val, y_train, y_val, input_length_train, input_length_val, label_length_train, label_length_val = train_test_split(X, y, input_lengths, label_lengths, test_size=0.2, random_state=42)

# Data generator
def data_generator(X, y, input_lengths, label_lengths, batch_size=16):  # Reduce batch size
    while True:
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            input_lengths_batch = input_lengths[i:i+batch_size]
            label_lengths_batch = label_lengths[i:i+batch_size]
            yield (
                {
                    'the_input': np.array(X_batch),
                    'the_labels': np.array(y_batch),
                    'input_length': np.array(input_lengths_batch),
                    'label_length': np.array(label_lengths_batch)
                },
                {'ctc': np.zeros([len(X_batch)])}
            )

train_gen = data_generator(X_train, y_train, input_length_train, label_length_train, batch_size=16)
val_gen = data_generator(X_val, y_val, input_length_val, label_length_val, batch_size=16)

steps_per_epoch = len(X_train) // 16
validation_steps = len(X_val) // 16

# Train the model
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=20, validation_data=val_gen, validation_steps=validation_steps)

# Save the model
model.save('/content/asr_model.h5')

Epoch 1/20
1218/1218 [==============================] - 140s 98ms/step - loss: 109.3919 - val_loss: 91.6167
Epoch 2/20
1218/1218 [==============================] - 116s 95ms/step - loss: 85.1368 - val_loss: 76.6770
Epoch 3/20
1218/1218 [==============================] - 117s 96ms/step - loss: 78.0080 - val_loss: 72.0407
Epoch 4/20
1218/1218 [==============================] - 117s 96ms/step - loss: 74.3242 - val_loss: 69.2113
Epoch 5/20
1218/1218 [==============================] - 116s 95ms/step - loss: 71.7828 - val_loss: 67.2666
Epoch 6/20
1218/1218 [==============================] - 115s 94ms/step - loss: 69.8211 - val_loss: 65.1654
Epoch 7/20
1218/1218 [==============================] - 114s 94ms/step - loss: 67.8757 - val_loss: 63.7890
Epoch 8/20
1218/1218 [==============================] - 114s 93ms/step - loss: 66.4806 - val_loss: 62.9161
Epoch 9/20
1218/1218 [==============================] - 114s 94ms/step - loss: 65.2911 - val_loss: 61.9611
Epoch 10/20
1218/1218 [=============

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 4s 4s/step
Predicted text: تفابات ک ندیده بود
Actual text: اتفاقاتی که ندیده بودم


In [7]:
# Redefine the inference model
inference_model = Model(inputs=input_data, outputs=y_pred)

# Load the weights from the saved model
inference_model.load_weights('/content/asr_model.h5')

# Load a sample and predict
sample_index = 25
sample_features = X[sample_index]
sample_input_length = np.array([sample_features.shape[0]])

sample_features = np.expand_dims(sample_features, axis=0)
sample_input_length = np.array([sample_features.shape[1]], dtype=np.int32)

# Predict with beam search decoding
preds = inference_model.predict(sample_features)
decoded_pred = tf.keras.backend.ctc_decode(preds, input_length=sample_input_length, greedy=False, beam_width=10, top_paths=1)[0][0]
decoded_pred = tf.keras.backend.get_value(decoded_pred)

# Ensure decoded_pred is a 1D array
decoded_pred = decoded_pred.flatten()

# Convert the decoded prediction to text
predicted_text = ''.join([index_map[i] for i in decoded_pred if i != -1])

# Actual text from the dataset
actual_text = df.iloc[sample_index]['transcript']

print(f"Predicted text: {predicted_text}")
print(f"Actual text: {actual_text}")

1/1 [==============================] - 5s 5s/step
Predicted text: دست بردار از ین کارهار
Actual text: دست بردار از این کارها
